In [2]:
import os
from IPython.display import display, Audio
from glob import glob
import random

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

import tensorflow as tf

In [3]:
data_dir = '/home/david/spiced_projects/final_project/data/recs'
input_samplerate = 22050
channels = 1
label_names = ['close', 'exit', 'files', 'mail', 'media', 'open', 'silence', 'web']
model_path = "/home/david/spiced_projects/final_project/models/voice_command_model.h5"
model = tf.keras.models.load_model(model_path)

In [4]:
def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels

def get_spectrogram(wave):
    spec = tf.signal.stft(wave, frame_length=300, frame_step=85)
    spec = tf.abs(spec)
    # add 'channels' dim back
    spec = spec[..., tf.newaxis]
    return spec

In [10]:
#filepathlist = glob("/home/david/spiced_projects/final_project/data/recs/*/*.wav")
#x = random.choice(filepathlist)

x = '/home/david/spiced_projects/final_project/data/test/exit/2023-07-30_13-25-14_101121.wav'
x = tf.io.read_file(x)
x, _ = tf.audio.decode_wav(x, desired_channels=channels, desired_samples=input_samplerate)
x = tf.squeeze(x, axis=-1)
waveform = x
x = get_spectrogram(x)
x = x[tf.newaxis,...]

prediction = model(x)

print(label_names[tf.argmax(tf.nn.softmax(prediction[0]))])

display(Audio(waveform, rate=input_samplerate))


exit


In [18]:
filepathlist = glob("/home/david/spiced_projects/final_project/data/test/files/*.wav")

for filepath in filepathlist:
    x = tf.io.read_file(filepath)
    x, _ = tf.audio.decode_wav(x, desired_channels=channels, desired_samples=input_samplerate)
    x = tf.squeeze(x, axis=-1)
    waveform = x
    x = get_spectrogram(x)
    x = x[tf.newaxis,...]

    prediction = model(x)

    print(label_names[tf.argmax(tf.nn.softmax(prediction[0]))])

files
files
exit
files
files
files
files
files
files
files


In [30]:
import sounddevice as sd
from time import sleep
import logging


logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)

#settings
fs = 22050
seconds = 1
n_channels = 1
sel_dtype = 'int16'
preferred_device = 'NexiGo N660P FHD Webcam: USB Audio (hw:1,0)'

if sd.query_devices(sd.default.device[0])['name'] == preferred_device:
    
    sd.check_input_settings(channels=n_channels, dtype=sel_dtype, samplerate=fs) #raises exception if the settings cannot be applied

    logging.info("Good to go...\n")

    print("Rec in 1 sec")
    sleep(1.0)
    print("GO!!!")
    
    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=n_channels, dtype=sel_dtype)
    sd.wait()  # Wait until recording is finished

else:
    logging.error("Wrong recording device!")

INFO: Good to go...



Rec in 1 sec
GO!!!


In [31]:
x = tf.convert_to_tensor(myrecording / 32767, tf.float32)
x = tf.squeeze(x, axis=-1)
waveform = x
x = get_spectrogram(x)
x = x[tf.newaxis,...]

prediction = model(x)

print(f"Predicted command: {label_names[tf.argmax(tf.nn.softmax(prediction[0]))]}")

display(Audio(waveform, rate=input_samplerate))

Predicted command: web
